In [1]:
import pandas as pd
import numpy as np
from blackjack import SimplifiedBlackjackMDP
from snake import SnakeMDP, hashable_state, array_state
import pickle
import random
import matplotlib.pyplot as plt

In [2]:
example_1 = hashable_state(
    (np.array([[0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.],
               [3., 1., 1., 1., 2.]]),
     'right')
)

example_2 = hashable_state(
    (np.array([[0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.],
               [0., 0., 0., 3., 0.],
               [0., 0., 0., 2., 0.],
               [0., 1., 1., 1., 0.]]),
     'up')
)
s = example_2

with open('snake_bl5_msl7_20221124.pkl', 'rb') as f:
    snake_mdp = pickle.load(f)

In [6]:
snake_policy

{((2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'up'): 'right',
 ((2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'down'): 'right',
 ((2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'left'): 'down',
 ((2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'right'): 'right',
 ((2.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'up'): 'right',
 ((2.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'down'): 'right',
 ((2.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'left'): 'down',
 ((2.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'right'): 'right',
 ((2.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'up'): 'right',
 ((2.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'down'): 'down',
 ((2.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'left'): 'down',
 ((2.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'right'): 'down',
 ((2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0), 'up'): 'right',
 ((2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0), 'down'): 'down',
 ((2.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0), 'left'): 'down',
 ((2.0,

In [5]:
snake_policy, snake_q = snake_mdp.Q_learning(gamma=0.9, epsilon=0.0001)

Beginning iteration i=1
	j=0
	j=100
	j=200
	j=300
	j=400
	j=500
	j=600
	j=700
	j=800
At iteration 1, max change in value: 1.00000
Beginning iteration i=2
	j=0
	j=100
	j=200
	j=300
	j=400
	j=500
	j=600
	j=700
	j=800
At iteration 2, max change in value: 0.90000
Beginning iteration i=3
	j=0
	j=100
	j=200
	j=300
	j=400
	j=500
	j=600
	j=700
	j=800
At iteration 3, max change in value: 0.81000
Beginning iteration i=4
	j=0
	j=100
	j=200
	j=300
	j=400
	j=500
	j=600
	j=700
	j=800
At iteration 4, max change in value: 0.72900
Beginning iteration i=5
	j=0
	j=100
	j=200
	j=300
	j=400
	j=500
	j=600
	j=700
	j=800
At iteration 5, max change in value: 0.65610
Beginning iteration i=6
	j=0
	j=100
	j=200
	j=300
	j=400
	j=500
	j=600
	j=700
	j=800
At iteration 6, max change in value: 0.00000


In [3]:
snake_mdp = SnakeMDP(board_length=3, max_snake_length=2)

Trying max snake length 1 with 9 base combos
100 states found...
200 states found...


In [ ]:
snake_policy, snake_value = mdp.policy_iteration(gamma=0.5, epsilon=0.0001, max_allowed_time=720)

# s = ((1.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'down')
# array_state(s)
# array_state(mdp.accessible_states(s, 'down')[0])

for s in random.sample(mdp.states, k=5):
    print(f'\nFor state s = \n{array_state(s)}')
    print(f'the recommended action is: {snake_policy[s]}')

In [ ]:
# take a state and visualize it

def plot(s):
    board, last_direction = array_state(s)
    plt.imshow(board, interpolation='none')
    plt.show()

In [ ]:

mdp = SimplifiedBlackjackMDP()


mdp._sample_state_layout()

for s in mdp.accessible_states((11, 3, 'hitting'), 'hit'):
    print(s, mdp.reward(s))
mdp.transition_model((19, 6, 'hitting'), 'hold', (19, 6, 'stand'))

blackjack_policy, blackjack_value = mdp.policy_iteration(gamma=0.99999, epsilon=1e-5)
blackjack_policy, blackjack_value = mdp.value_iteration(gamma=0.99999, epsilon=1e-5)

policy_visualization = pd.DataFrame(
    index=pd.Index(range(2, 12), name='Dealer Value'),
    columns=pd.Index(range(2, 22), name='Player Value'),
    dtype='string'
)
for i in range(2, 12):
    for j in range(2, 22):
        policy_visualization.loc[i, j] = blackjack_policy[(j, i, 'hitting')]

policy_visualization.iloc[:, 5:15]

value_visualization = pd.DataFrame(
    index=pd.Index(range(2, 12), name='Dealer Value'),
    columns=pd.Index(range(2, 22), name='Player Value'),
    dtype='float'
)
for i in range(2, 12):
    for j in range(2, 22):
        value_visualization.loc[i, j] = np.round(blackjack_value[(j, i, 'hitting')], 1)

value_visualization.iloc[:, 5:15]
